In [ ]:
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas as pd 
import numpy as np 
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from matplotlib.image import imread
from transformers import BertTokenizer,TFBertModel

In [ ]:
train_df=pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
train_pro_df=pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")

In [ ]:
train_df

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.sample()
train_df["lower_test"]=train_df["text"].str.lower()
# =texts.str.lower

In [ ]:
# e=train_df["lower_test"]

In [ ]:
# drop the new column created in last cell
# df.drop(["text_lower"], axis=1, inplace=True)
import string

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

train_df["text_wo_punct"] = train_df["text"].apply(lambda text: remove_punctuation(text))
train_df.head()

In [ ]:
from nltk.corpus import stopwords
",".join(stopwords.words("english"))

In [ ]:
stopwordsss=set(stopwords.words("english"))
def stopword12(text):
    return " ".join([word for word in str(text).split() if word is not stopwordsss])
train_df["stop_words1"]=train_df["text_wo_punct"].apply(lambda text: stopword12(text))
train_df.head()
# r=lambda e : d

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

train_df["text_wo_stop"] = train_df["stop_words1"].apply(lambda text: remove_stopwords(text))
train_df.head()

In [ ]:
train_df

In [ ]:
from collections import Counter
cnt = Counter()
for text in train_df["lower_test"].values:
    for word in text.split():
        cnt[word] += 1
        
collect_c=cnt.most_common()

In [ ]:
collect_c[:10]

In [ ]:
freqwords=set([w for (w,wn) in cnt.most_common(10)])
def common_word(text):
    return "  ".join([w for w in str(text).split() if w is not freqwords])
train_df["common_word"]=train_df["text_wo_stop"].apply(lambda text : common_word(text))

In [ ]:
train_df["generated"].nunique()

# Text modeling

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count_vec=CountVectorizer()
count_vec.fit(train_df["common_word"])
print(count_vec.vocabulary_)
vect=count_vec.transform(train_df["common_word"])

In [ ]:
print(vect.shape)
print(vect.toarray())

# tfidfvectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vector_t=TfidfVectorizer()
vector_t.fit(train_df["common_word"])
# print(vector_t.vocabulary_)
tra_vec=vector_t.transform(train_df["common_word"])
print(tra_vec.shape)
print(tra_vec.toarray())

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load your data
# df = pd.read_csv('your_dataset.csv')
texts = train_df['common_word']
labels = train_df['generated']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a classifier
classifier = LogisticRegression()
classifier.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100)  # Adjust the number of components as needed
tfidf_reduced = svd.fit_transform(X_train_tfidf)
print(tfidf_reduced.shape)
plt.figure(figsize=(10, 6))
plt.scatter(tfidf_reduced[:, 0], tfidf_reduced[:, 1], alpha=0.5)
plt.title('2D Visualization of Reduced TF-IDF Matrix')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
mk=KMeans()
# mk.fit(X_train_tfidf)
clust_sk=mk.fit_transform(X_train_tfidf)
print(clust_sk.shape)
plt.figure(figsize=(10, 6))
plt.scatter(clust_sk[:, 0], clust_sk[:, 1], alpha=0.5)
plt.title('2D Visualization of Reduced TF-IDF Matrix')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()

In [ ]:
train_pro_df.head()


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

text_data = [" ".join(map(str, row)) for row in X_train_tfidf]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(text_data)

# Pad sequences to ensure uniform length
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='pre')

# Split sequences into input and output
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]

# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length - 1),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(len(tokenizer.word_index) + 1, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Predict the next word



In [ ]:
input_text = "Cars have been around"
input_sequence = tokenizer.texts_to_sequences([input_text])
input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length - 1, padding='pre')
predicted_index = model.predict(input_sequence, verbose=0)[0]

predicted_index_scalar = predicted_index[0].item()
# Round the predicted index to the nearest integer
predicted_index_rounded = int(round(predicted_index_scalar))

# Convert index back to word
predicted_word = tokenizer.index_word.get(predicted_index_rounded, 'Unknown')

print("Input:", input_text)
print("Predicted Next Word:", predicted_word)

In [ ]:
tokenizer
test_on_sequence
pad
embed


In [ ]:
tokenize=tokenizer()
text=tokenize.fit(X_test)
text=test_on_sequence
test.paddding
layers.Embedding